# Project :6 Working with Structured Data: Classification

In the realm of maritime operations, accurate detection and classification of underwater objects play a pivotal role in ensuring the safety and efficiency of various endeavours, ranging from underwater exploration to resource extraction. The Sonar dataset, comprising 208 observations, provides a comprehensive repository of sonar chirp returns, offering insights into the characteristics of submerged surfaces. With 60 input variables delineating the strength of returns at different angles, this dataset presents a formidable challenge: the binary classification of rocks and metal cylinders.By leveraging advanced predictive modelling techniques, our aim is to develop a robust solution capable of accurately discerning between these two distinct entities, thereby enhancing operational efficiency, and minimizing risks associated with underwater activities.

        

#### Data Set Description:

The data set contains signals obtained from a variety of different aspect angles, spanning 90 degrees for the cylinder and 180 degrees for the rock. Each pattern is a set of 60 numbers in the range 0.0 to 1.0. Each number represents the energy within a particular frequency band, integrated over a certain period. The label associated with each record contains the letter "R" if the object is a rock and "M" if it is a mine (metal cylinder).  

**Dataset : sonar.csv**

Create a first step document that lists the output of your exploratory analysis, any issues, or problems you may see with data that need follow-up, and some basic descriptive analysis that you think highlights important outcomes/findings from the data. Based on your findings, the next level of analysis will be charted out. Build a predictive base model and use hyperparameter tuning to further optimize the accuracy of the predictions.  

**Initial Guidelines:**

1.	Ensure to follow to Use Id’s provided by UNext for naming file as conventions.
2.	Create GitHub account and submit the GitHub link.

### Software Engineering aspect:  

Utilize software engineering aspects while building Machine learning model using modular programming principles to organize your code into reusable functions or classes to enhance readability, maintainability, and collaboration.

### General Instructions 

- The cells in the Jupyter notebook can be executed any number of times for testing the solution
- Refrain from modifying the boilerplate code as it may lead to unexpected behavior 
- The solution is to be written between the comments `# code starts here` and `# code ends here`
- On completing all the questions, the assessment is to be submitted on moodle for evaluation
- Before submitting the assessment, there should be `no error` while executing the notebook. If there are any error causing code, please comment it.
- The kernel of the Jupyter notebook is to be set as `Python 3 (ipykernel)` if not set already
- Include imports as necessary
- For each of the task, `Note` section will provide you hints to solve the problem.
- Do not use `PRINT` statement inside the `Except` Block. Please use `return` statement only within the except block

#### **Utilize software engineering aspects while building the  model using modular programming principles to organize your code into reusable functions or classes to enhance readability, maintainability, and collaboration.**

In [ ]:
# Required imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report,roc_auc_score

### Task 1: Load the dataset and perform preliminary EDA with key observations and insights- (weightage - 20 marks)

#### T1.1: Load the sonar dataset using try and except blocks.          (weightage - 2 marks) (AE)        

#### NOTE:
- The `read_csv` method in Pandas allows you to read csv files and convert them into a DataFrame, which is a two-dimensional tabular data structure in Pandas.
- Do not use `PRINT` statement inside the `Except` Block. Please use `return` statement only within the except block

In [ ]:
def load_the_dataset():
    try:
    # code starts here
      df_sonar=pd.read_csv('sonar1.csv')
    # code ends here
      return df_sonar
    except:
        return "File not found. Please check the file path."


In [ ]:
# store the result of the dataset
df_sonar =load_the_dataset()
print(df_sonar)

#### T1.2: What is the distribution of target variable? (Mention in percentage)(weightage - 2 marks)  (AE)             

### NOTE:

* Design a method `target_class` to analyze the distribution of a target variable ("class") within a DataFrame (df). 
* Count the occurrences of each unique value in the 'class' column of the DataFrame df using the `value_counts` method
* Normalizes the counts to obtain proportions instead of counts.
* Scale the proportions to percentages.

In [ ]:
def target_class(df):
  #code starts here
    target_dist = df['class'].value_counts(normalize=True)*100
    #code ends
    return target_dist

In [ ]:
# store the result
target_distribution = target_class(df_sonar)
print(target_distribution)

#### T1.3: Remove the unnecessary column. (weightage - 2 marks)               (AE)

#### Note:
* Unnamed: 0 is the unecessary column.
* Use `drop` method of the dataframe to drop the unnecessary column.

In [ ]:
def unecessary_col(df):
  #code starts here
    try:
        dropped = df.drop(columns=['Unnamed: 0'])
        return dropped
    except:
        return "Failed to drop the column.Check again"
  #code ends here

In [ ]:
#Store the result into a new dataframe
df_sonar_updated = unecessary_col(df_sonar)
print(df_sonar_updated)

#### T1.4: Check missing values in the data in terms of percentage and do missing value treatment.  (weightage - 2 marks)       (AE)

#### NOTE:
- Find the percentage of missing values in the data by dividing the total number of missing values by the total number of rows and multiplying by 100, you will get the percentage of missing values for each column. 
- Use `isnull().sum()` to calculate the total number of missing values in each column and `shape[0]` to get the total number of rows in the DataFrame.
- Do not use `PRINT` statement inside the `Except` Block. Please use `return` statement only within the except block

In [ ]:
def missing_value_check(df):
    #Code starts here
    try:
        # Check for missing values
        missing_values = df.isnull().sum()
        missing_percentage = (missing_values / df.shape[0])* 100
        return  missing_percentage
    except Exception as e:
        return 'An error occurred'
    # Code ends here   

In [ ]:
missing_value_check(df_sonar_updated)

#### NOTE:
* For treating the missing value first define the list of columns containing the names of the columns with missing values which will be treated. 
* Iterating over each column name in the columns list fill the missing values in the dataset for that column with the median value of that column. 

In [ ]:
# Missing value treatment
def missing_value_treatment(df):
    columns=['Attribute7','Attribute12']
    for i in columns:
        df[i]=df[i].fillna(df[i].median())
    return df  

In [ ]:
#Check if missing values are treated
df_sonar_updated = missing_value_treatment(df_sonar_updated)
print(df_sonar_updated.isnull().sum())

#### T1.5: Visualize the correlation matrix using a heatmap for first 4 input variables. What is the Pearson correlation coefficient between the Attribute1 and Attribute 2? (Bivariate analysis) (weightage - 3 marks)               (AE and ME)

#### NOTE:
* Define a method called `correlation_matrix` to calculate the Pearson correlation coefficient between two specific columns   ('Attribute 1' and 'Attribute 2') in a DataFrame (df).
* Use the `iloc` method to select only the columns corresponding to 'Attribute 1' and 'Attribute 2' from the DataFrame df.
* Calculate the Pearson correlation coefficient using the corr() method on the selected columns.

In [ ]:
# Pearson correlation coefficient for Attribute 1 and Attribute 2
def correlation_matrix(df):
    correlation = df.iloc[:,[0,1]].corr()
    return correlation

In [ ]:
Correlation_Attribute_1_2 = correlation_matrix(df_sonar_updated)
print(Correlation_Attribute_1_2)

#### NOTE:
* Generate a heatmap visualization of the correlation matrix for the specified subset of columns in the DataFrame 
* Select the first four columns (indexed 0, 1, 2, and 3) of the DataFrame using integer-based indexing. 
* Using seaborn `heatmap` method, generate a heatmap plot of the correlation matrix.
* Using `annot` parameter, display correlation coefficients inside each cell.

In [ ]:
#Visualize using Heatmap
sns.heatmap(df_sonar_updated.iloc[:,[0,1,2,3]].corr(),annot=True)

#### T1.6. Perform PCA (Principal Component Analysis) on the input variables. Display the cumulative variance array along with the number of principal components required to capture 90% of information. How many principal components are required to capture 90% of the information?  (AE)

### Note

* Define a method called `pca_inputs` to perform Principal Component Analysis (PCA) on the input DataFrame (df) for the first 60 columns.
* Create a copy of the first 60 columns of the DataFrame df.* and assign it to a new DataFrame `df2`.
* Initialize a StandardScaler object (sc) to standardize the data.
* Standardize the data in `df2` using sc.fit_transform(df2), and store the standardized data in a new DataFrame dfsc.
* Convert dfsc back to a DataFrame with column names from df2 using pd.DataFrame(dfsc, columns=df2.columns)
* Initialize a PCA object (finalpca) for performing PCA.
* Perform PCA on the standardized data dfsc using `finalpca.fit_transform(dfsc)` and store the transformed data in finaldf.
* Obtain the explained variance ratio of each principal component using finalpca.explained_variance_ratio_
* Calculate the number of principal components needed to explain at least 90% of the variance by iterating through the cumulative sum of explained variances (np.cumsum(variation)).

In [ ]:
def pca_inputs(df):
    df2 = df.iloc[:,:60].copy()
    sc = StandardScaler()
    dfsc = sc.fit_transform(df2)
    dfsc = pd.DataFrame(dfsc,columns=df2.columns)
    finalpca = PCA()
    finaldf = finalpca.fit_transform(dfsc)
    variation = finalpca.explained_variance_ratio_
    count = 0
    for i in np.cumsum(variation):
      if i <= 0.90:
        count+=1
    return np.cumsum(variation),count

In [ ]:
# Print the cumulative variance and specify the number of principal components that capture 90% of information
cumulative_variance,count = pca_inputs(df_sonar_updated)
print(f'Cumulative variance :{cumulative_variance}')
print(f'Number of principal components that capture 90% of information :{count}')

#### T1.7: Data Transformation:  Apply normalization technique (standard scalar). (weightage - 3 marks)        (AE)       

### Note

* Define a method called `normalize` to normalize the first 60 columns of a DataFrame (df).
* Use the `iloc` method to select the first 60 columns of the DataFrame df, and apply the StandardScaler to standardize these columns.

In [ ]:
def normalize(df):
    df_norm = pd.DataFrame(StandardScaler().fit_transform(df.iloc[:,:60]),columns=df.iloc[:,:60].columns)
    return df_norm

In [ ]:
# Save the standardized data
df_normalized = normalize(df_sonar_updated)
print(df_normalized)

#### T1.8: Handling categorical features: Apply label encoding technique to convert categorical variable into numerical. (weightage - 2 marks) (AE)

#### NOTE:

* Define a method called `label_encode` to perform label encoding on a categorical column ('class') in a DataFrame (df).
* Initialize a LabelEncoder object (encoder) for encoding categorical labels.
* Use encoder.fit_transform(df[['class']]) to fit and transform the 'class' column into encoded labels. Wrap the result in a DataFrame using pd.DataFrame(...) and assign it to encoded_y.

In [ ]:
def label_encode(df):
    encoder = LabelEncoder()
    encoded_y = pd.DataFrame(encoder.fit_transform(df[['class']]),columns=df[['class']].columns)
    return encoded_y


In [ ]:
# Apply label encoding to class variable
encoded_class = label_encode(df_sonar_updated)
print(encoded_class)

#### T1.9: Save the cleaned dataset (.xlsx file) by setting the index=False in your GitHub repository for model building process. (This task is for maintaining the version control of datasets)        

#### NOTE:
* Concatenate two DataFrames `df_normalized` and `encoded_class` horizontally along the `columns` using the `concat` method
* Using `to_excel` method and setting `index` parameter to `False` save the final dataset in __EXCEL [.xlsx]__ format

In [ ]:
df_final = pd.concat([df_normalized,encoded_class],axis=1)
df_final.to_excel("df_final.xlsx",index=False)

#### Refer to the Github document from Lumen to create the repository and steps to commit 
#### Add your Github repository link below 

In [ ]:
# save it in github repository

### Task 2: Build a Neural Network Predictive Model with Grid Search (weightage - 30 marks)               

#### T2.1: Load the cleaned dataset and divide it into predictor and target values (X & y) (weightage – 5 marks) (AE)

#### NOTE:

* Define a method called `load_the_dataset` to load a cleaned dataset from an Excel file named 'df_final.xlsx'.
* Use the `pd.read_excel` function to read the Excel file and store the data in a DataFrame called cleaned_df.

In [ ]:
# load the cleaned data
def load_the_cleaned_dataset():
        cleaned_df = pd.read_excel('df_final.xlsx')
        return cleaned_df

In [ ]:
cleaned_df=load_the_dataset()

- Separate independent features and target variable

#### NOTE

* Define a method called `separate_data_and_target` to separate the independent features and the target variable from a DataFrame (df).
* Use the `drop` method on the DataFrame df to create a new DataFrame `X` containing only the independent features. Drop the `class` column along the columns axis (axis=1).
* Create a Series `y` containing the target variable by selecting only the `class` column from the original DataFrame df.

In [ ]:
# Separate independent features and target variable
def separate_data_and_target(df):
    X = df.drop(['class'], axis=1)
    y = df['class']
    return X,y

In [ ]:
# dependent : X, independent : y
X, y = separate_data_and_target(cleaned_df)
print(X.head())

In [ ]:
print(y.head())

#### T2.2: Split the dataset into train and test in the ratio of 80:20. (weightage – 5 marks) (ME)

#### NOTE
* Define a method called `split_into_train_and_test_normalize_features` to split the features (X) and target variable (y) into training and testing sets, and normalize the features.
* Use the `train_test_split` function to split the features (X) and target variable (y) into training and testing sets. Specify the test size as 20% (test_size = .2) and set a random state for reproducibility (random_state=0).

In [ ]:
from sklearn.model_selection import train_test_split
def split_into_train_and_test_normalize_features(X,y):
# Splitting dataset to train and test sets 80% train and 20% test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=0)
    return X_train, X_test,y_train,y_test

In [ ]:
# split into training and testing
X_train, X_test,y_train,y_test=split_into_train_and_test_normalize_features(X,y)

#### Build NN model on training data.Perform hyperparameter tuning using Grid Search. What are the epochs, batch size, and optimizers were selected using grid search?

**Model_versioning**

Save the model as ‘NN_model’ to a version control system GitHub using git commands for collaboration, tracking changes, and ensuring transparency in model development. --- (weightage 20 marks ) (ME)

#### NOTE

* Define a method called `create_baseline` to create a baseline neural network model.
* Inside the method:
    * Create a Sequential model using Sequential() from Keras.
    * Add a Dense layer with 60 units (assuming 60 input features), specify input_dim=60, use 'normal' kernel initializer, and set activation='relu'.
    * Add another Dense layer with 1 unit for binary classification, use 'normal' kernel initializer, and set activation='sigmoid'.
    * Compile the model using model.compile() with 'binary_crossentropy' as the loss function, 'adam' optimizer, and include 'accuracy' as a metric.
* Create Pipeline and Grid Search:
    * Create a pipeline using Pipeline from sklearn.pipeline.
    * Include the KerasClassifier within the pipeline, specifying the build function as create_baseline and set verbose=0.
    * Define the grid search parameters in a dictionary param_grid, such as different batch sizes and epochs.
* Perform Grid Search:
    * Initialize a StratifiedKFold with the desired number of splits (e.g., StratifiedKFold(n_splits=10, shuffle=True)).
    * Use GridSearchCV with the pipeline, parameter grid (param_grid), and cross-validation strategy (cv=kfold) to perform grid search.
    * Fit the grid search object to your training data (X_train, y_train) to find the best model parameters.

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.pipeline import Pipeline

def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create pipeline
pipeline = Pipeline([('mlp', KerasClassifier(build_fn=create_baseline, verbose=0))])

# define the grid search parameters
param_grid = {
    'mlp__batch_size': [5, 10, 20],
    'mlp__epochs': [50, 100, 150],
}

# perform grid search
kfold = StratifiedKFold(n_splits=10, shuffle=True)
grid = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=kfold)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


The results indicate the performance of the baseline model and its variations with different hyperparameters.

#### NOTE
* Utilizes Keras and scikit-learn to create and train a neural network model with the specified parameters.
* Train the model on the training data

In [ ]:
best_batch_size = 5
best_epochs = 150
model = KerasClassifier(build_fn=create_baseline, epochs=best_epochs, batch_size=best_batch_size, verbose=0)
# Retrain the model on the entire training set
model.fit(X_train, y_train)

### Task 3: Evaluate the performance of the model using the right evaluation metrics. (weightage - 25 marks)

#### T3.1: Bring the ‘NN-Model’ from a GitHub using git commands and evaluate the model.(ME)

#### T3.2:  Evaluate the NN model with evaluation metrics accuracy, precision, recall, f1 score and roc auc score using sklearn library.(weightage -12 marks) (AE)

#### NOTE
* Create a method called `ROC_AUC` to calculate the ROC-AUC score for a given model's predictions on test data.
* Accept three parameters:
    model: The trained model for which you want to calculate ROC-AUC.
    X_test: The feature matrix of the test data.
    y_test: The true labels of the test data.
* Calculate ROC-AUC Score:
* Use the `predict_proba` method of the model to get the predicted probabilities of the positive class (class 1) for the test data. Ensure to select the probabilities of the positive class.


In [2]:
def ROC_AUC(model, X_test, y_test):

    ypred_prob = model.predict_proba(X_test, verbose=0)[:, 1]
    # code starts here
    auc_score = roc_auc_score(y_test, ypred_prob)

    # code ends here
    return auc_score

In [3]:
print(ROC_AUC(model,X_test,y_test))

NameError: name 'ypred_prob' is not defined

#### Note
* Create a method called `evaluate_classification` to evaluate a classification model's performance on test data.
* Accept three parameters:
    model: The trained classification model to evaluate.
    X_test: The feature matrix of the test data.
    y_true: The true labels of the test data.
* Calculate Evaluation Metrics
* Use the predict method of the model to get the predicted labels for the test data.
* Calculate evaluation metrics such as accuracy, precision, recall, and F1 score using functions like accuracy_score, precision_score, recall_score, and f1_score from sklearn.metrics.
* Set `verbose=0` to suppress output

In [ ]:
def evaluate_classification(model,X_test,y_true):
    y_pred = model.predict(X_test, verbose=0)

     # code starts here
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    metrics = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }
    return accuracy,precision,recall,f1

In [ ]:
evaluate_classification(model,X_test,y_test)

After hyperparameter tuning,it has been observed that the parameters 'mlp__batch_size' of 5, 'mlp__epochs'of 150 has given a good accuracy score. The model is then tested on the test dataset and is observed that it works well with a precision score of 0.94,recall of 0.81 and a f1score of 0.871. This classification model can be used to distinguish the rocks from metal cylinders.

#### Using Lime/SHAP libraries, explain the prediction of your  model and give inferences. (weightage- 5marks) (ME)

#### T3.4 Implement the unit test case and deploy a model using Flask / Streamlit. (weightage-15 marks) (ME)

#### NOTE

* Define a Sequential model keras_model using `Sequential()` from Keras.
* Add layers to `keras_model` to match the architecture of your trained model:
* Add a Dense layer with 60 units, input dimension of 60, 'normal' kernel initializer, and 'relu' activation.
* Add another Dense layer with 1 unit for binary classification, 'normal' kernel initializer, and 'sigmoid' activation.
* Load Trained Model Weights.
* Use keras_model.set_weights(model.model.get_weights()) to load the weights from the trained model model into your keras_model. This assumes that model is an instance of KerasClassifier or KerasRegressor from scikit-learn.

In [ ]:
# Save the Keras model to an HDF5 file
from keras.models import Sequential, save_model

# Recreate the Keras model
keras_model = Sequential()
keras_model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
keras_model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

# Load the weights from the best KerasClassifier model
keras_model.set_weights(model.model.get_weights())

# Save the Keras model to an HDF5 file
save_model(keras_model, 'model.h5')

Final Submission guidelines: 
1.	Download the Jupyter notebook in the format of html. 
2.	Upload it in the lumen (UNext LMS)
3.	Take a screenshot of T15 (Deployment) and upload it in the lumen (UNext LMS)
4.	Summarized PPT/ PDF prepared in Task 4 to be uploaded in the lumen (UNext LMS)

-------------------------------------------------- **ASSESSMENT ENDS HERE** ---------------------------------------------------------